In [ ]:
import numpy as np

import json
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
import keras.backend as K
import tensorflow as tf

In [ ]:
with open('data/sentipolc_char_index.json', 'r') as f:
    word_index = json.load(f)

In [ ]:
with open('data/chars.tsv', 'w') as f:
    for w,_ in word_index.items():
        f.write('{}\n'.format(w))

In [ ]:
words = [w for w,_ in word_index.items()]

In [ ]:
data = np.load('data/sentipolc_char_seq.npz')

x_train = data['x_train']
y_train = data['y_train']

x_test = data['x_test']
y_test = data['y_test']

Check the max length of the text

In [ ]:
max_len_seq = max([len(x) for x in x_train])
print('max len seq {}'.format(max_len_seq))
max_idx = max(word_index.values())
print('max id {}'.format(max_idx))

In [ ]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=max_len_seq, padding='post')
x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len_seq, padding='post')

In [ ]:
x_train_pad.shape

# Define metrics

In [ ]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.
     Computes the precision, a metric for multi-label classification of
     how many selected items are relevant.
      """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
def recall(y_true, y_pred):
    """Recall metric.
 
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split_train_val = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

for train, test in split_train_val.split(x_train_pad, y_train):
    x_train_pad_split, y_train_pad_split = x_train_pad[train], y_train[train]
    x_val_pad_split, y_val_pad_split = x_train_pad[test], y_train[test]

In [ ]:
callbacks = [
    TensorBoard(log_dir='./logs_char_lstm', histogram_freq=1, batch_size=128, embeddings_freq=1, embeddings_metadata=words),
    ReduceLROnPlateau(patience=2, verbose=1)
]

In [ ]:
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_idx+1, output_dim=64, input_length=max_len_seq))
    model.add(LSTM(64,  dropout=0.0, recurrent_dropout=0.0, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
!rm -rf logs_word_lstm
!mkdir -p logs_word_lstm

In [ ]:
history = model.fit(x_train_pad_split, y_train_pad_split, validation_data=(x_val_pad_split, y_val_pad_split),
                    batch_size=32, epochs=10, callbacks=callbacks)

In [ ]:
del model

## Evaluate on the test set

In [ ]:
callbacks = [
    TensorBoard(histogram_freq=0, batch_size=128, embeddings_freq=1, embeddings_metadata=words),
]

In [ ]:
model = build_model()

history = model.fit(x_train_pad, y_train, batch_size=128, epochs=6, callbacks=callbacks)

In [ ]:
loss, acc = model.evaluate(x_test_pad, y_test)

In [ ]:
print('loss {}, acc {}'.format(loss, acc))

In [ ]:
#model.save('models/sentipolc_char_lstm.hdf5')

## Confusion matrix

In [ ]:
from sklearn import metrics

In [ ]:
predictions = model.predict_classes(x_train_pad, batch_size=128)

In [ ]:
tn, fp, fn, tp = metrics.confusion_matrix(y_train, predictions).ravel()

In [ ]:
recall = tp/(tp+fp)
precision = tp/(tp+fn)
f1 = metrics.f1_score(y_train,predictions)

print ('precision: {}'.format(precision), '\nrecall: {}'.format(recall), '\nf1: {}'.format(f1))